In [102]:
from entities import *
from constants import *
from simulations import *
from tournament_roster import *
from constants import TEAMS_DATA, STAT_EFFECTS

import pickle
import numpy as np
import pandas as pd
import openpyxl as xl

from pathlib import Path

LOAD = True
PRECOMPUTED_GAMES = Path.cwd() / 'data' / 'pre_computed_games.pkl'
BLANK_BRACKET = Path.cwd() / 'resources' / 'blank_bracket.xlsx'
OUTPUT = Path.cwd() / 'output'

In [5]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

if LOAD:
    with open(PRECOMPUTED_GAMES, 'rb') as f:
        games = pickle.load(f)

In [23]:
wb = xl.load_workbook('resources/blank_bracket.xlsx', data_only=True)

tournament_roster = [teams[wb['Teams'][f'C{i}'].value] for i in range(2, 66)]
tournament_roster

[<Team: Abilene Christian Record: 10-10>,
 <Team: Air Force Record: 3-17>,
 <Team: Akron Record: 17-4>,
 <Team: Alabama Record: 14-6>,
 <Team: Alabama A&M Record: 11-9>,
 <Team: Alabama State Record: 6-14>,
 <Team: Albany (NY) Record: 7-15>,
 <Team: Alcorn State Record: 3-16>,
 <Team: American Record: 13-9>,
 <Team: Appalachian State Record: 14-9>,
 <Team: Arizona Record: 21-0>,
 <Team: Arizona State Record: 11-10>,
 <Team: Arkansas Record: 16-5>,
 <Team: Arkansas State Record: 13-9>,
 <Team: Arkansas-Pine Bluff Record: 7-13>,
 <Team: Army Record: 9-13>,
 <Team: Auburn Record: 14-7>,
 <Team: Austin Peay Record: 14-6>,
 <Team: Ball State Record: 7-13>,
 <Team: Baylor Record: 11-9>,
 <Team: Bellarmine Record: 8-13>,
 <Team: Belmont Record: 19-3>,
 <Team: Bethune-Cookman Record: 9-11>,
 <Team: Binghamton Record: 5-18>,
 <Team: Boise State Record: 13-8>,
 <Team: Boston College Record: 9-11>,
 <Team: Florida Record: 15-6>,
 <Team: Duke Record: 19-1>,
 <Team: Bradley Record: 14-8>,
 <Team: B

In [ ]:
def round_of_games(team_list: list, n: int=20, summary=False):
    winners, losers = [], []
    schools = []
    records = []
    analyses = []

    for i in range(0, len(team_list), 2):
        game = simulate_n_games(team_list[i], team_list[i+1], n, summary=summary)
        winners.append(teams[game['Winner']].name)
        losers.append(teams[game['Loser']].name)
        analyses.append(game['Analysis'])
        schools.append([game['Team 1'], game['Team 2']])
        records.append([game['Win Count 1'], game['Win Count 2']])
    
    return {'Winners': winners, 
            'Losers': losers, 
            'Analysis': analyses, 
            'Schools': schools,
            'Records': records
        }

def first_round(region: dict, n: int=20):
    first_round_teams = [teams[v] for v in region.values()]
    return round_of_games(first_round_teams, n=n)

# Precalculated Games

In [219]:
def precomputed_games(team_1, team_2):
    for game in games:
        game_players = [game['Winner'], game['Loser']]
        if (team_1 in game_players) and (team_2 in game_players):
            
            output = {
                'T1': game['Team 1'].name,
                'T1%': game['Team 1 Win Pct'],
                'T2': game['Team 2'].name,
                'T2%': game['Team 2 Win Pct'],
                'W': game['Winner'],
                'Analysis': game['Analysis']
            }

            return output

def precomputed_games_from_list(l):
    for i in range(0, len(l), 2):
        yield precomputed_games(l[i], l[i+1])

def parse_games(outcomes):
    round_t1 = [t['T1'] for t in outcomes]
    round_t2 = [t['T2'] for t in outcomes]
    round_t1_pct = [t['T1%'] for t in outcomes]
    round_t2_pct = [t['T2%'] for t in outcomes]
    round_analysis = [t['Analysis'] for t in outcomes]
    round_winner = [t['W'] for t in outcomes]
    return round_t1, round_t2, round_t1_pct, round_t2_pct, round_analysis, round_winner


def precomputed_region(roster_list):
    first_round = parse_games(list(precomputed_games_from_list(roster_list)))
    second_round = parse_games(list(precomputed_games_from_list(first_round[0])))
    third_round = parse_games(list(precomputed_games_from_list(second_round[0])))
    fourth_round = parse_games(list(precomputed_games_from_list(third_round[0])))
    return first_round, second_round, third_round, fourth_round

# Making The Spreadsheet

In [220]:
import shutil

new_bracket = OUTPUT / 'Bracket1.xlsx'
shutil.copyfile(BLANK_BRACKET, new_bracket)

WindowsPath('c:/Users/amgal/Documents/Data Science/KingfisherCourt/output/Bracket1.xlsx')

In [253]:
region_team_1 = {}
region_team_2 = {}
region_scores_1 = {}
region_scores_2 = {}
region_analysis = {}
region_winners = {}

# Rounds of 64 through Elite Eight
for region, region_name in zip([south, west, east, midwest], ['South', 'West', 'East', 'Midwest']):
    region_team_1[region_name] = [b[0] for b in precomputed_region(list(region.values()))]
    region_team_2[region_name] = [b[1] for b in precomputed_region(list(region.values()))]
    region_scores_1[region_name] = [b[2] for b in precomputed_region(list(region.values()))]
    region_scores_2[region_name] = [b[3] for b in precomputed_region(list(region.values()))]
    region_analysis[region_name] = [b[4] for b in precomputed_region(list(region.values()))]
    region_winners[region_name]  = [b[5] for b in precomputed_region(list(region.values()))]

team_1_col = region_team_1['South'][0] + region_team_1['West'][0] + region_team_1['East'][0] + region_team_1['Midwest'][0] + \
             region_team_1['South'][1] + region_team_1['West'][1] + region_team_1['East'][1] + region_team_1['Midwest'][1] + \
             region_team_1['South'][2] + region_team_1['West'][2] + region_team_1['East'][2] + region_team_1['Midwest'][2] + \
             region_team_1['South'][3] + region_team_1['West'][3] + region_team_1['East'][3] + region_team_1['Midwest'][3]

team_2_col = region_team_2['South'][0] + region_team_2['West'][0] + region_team_2['East'][0] + region_team_2['Midwest'][0] + \
             region_team_2['South'][1] + region_team_2['West'][1] + region_team_2['East'][1] + region_team_2['Midwest'][1] + \
             region_team_2['South'][2] + region_team_2['West'][2] + region_team_2['East'][2] + region_team_2['Midwest'][2] + \
             region_team_2['South'][3] + region_team_2['West'][3] + region_team_2['East'][3] + region_team_2['Midwest'][3]

region_scores_1 = region_scores_1['South'][0] + region_scores_1['West'][0] + region_scores_1['East'][0] + region_scores_1['Midwest'][0] + \
                  region_scores_1['South'][1] + region_scores_1['West'][1] + region_scores_1['East'][1] + region_scores_1['Midwest'][1] + \
                  region_scores_1['South'][2] + region_scores_1['West'][2] + region_scores_1['East'][2] + region_scores_1['Midwest'][2] + \
                  region_scores_1['South'][3] + region_scores_1['West'][3] + region_scores_1['East'][3] + region_scores_1['Midwest'][3]

region_scores_2 = region_scores_2['South'][0] + region_scores_2['West'][0] + region_scores_2['East'][0] + region_scores_2['Midwest'][0] + \
                  region_scores_2['South'][1] + region_scores_2['West'][1] + region_scores_2['East'][1] + region_scores_2['Midwest'][1] + \
                  region_scores_2['South'][2] + region_scores_2['West'][2] + region_scores_2['East'][2] + region_scores_2['Midwest'][2] + \
                  region_scores_2['South'][3] + region_scores_2['West'][3] + region_scores_2['East'][3] + region_scores_2['Midwest'][3]

region_analysis = region_analysis['South'][0] + region_analysis['West'][0] + region_analysis['East'][0] + region_analysis['Midwest'][0] + \
                  region_analysis['South'][1] + region_analysis['West'][1] + region_analysis['East'][1] + region_analysis['Midwest'][1] + \
                  region_analysis['South'][2] + region_analysis['West'][2] + region_analysis['East'][2] + region_analysis['Midwest'][2] + \
                  region_analysis['South'][3] + region_analysis['West'][3] + region_analysis['East'][3] + region_analysis['Midwest'][3]

# Final Four
south_west = precomputed_games(region_winners['South'][3][-1], region_winners['West'][3][-1])
east_midwest = precomputed_games(region_winners['East'][3][-1], region_winners['Midwest'][3][-1])

team_1_col += [south_west['T1'] , east_midwest['T1']]
team_2_col += [south_west['T2'] , east_midwest['T2']]
region_scores_1 += [south_west['T1%'] , east_midwest['T1%']]
region_scores_2 += [south_west['T2%'] , east_midwest['T2%']]
region_analysis += [south_west['Analysis'] , east_midwest['Analysis']]

# Final
final = precomputed_games(south_west['W'], east_midwest['W'])

team_1_col += [final['T1']]
team_2_col += [final['T2']]
region_scores_1 += [final['T1%']]
region_scores_2 += [final['T2%']]
region_analysis += [final['Analysis']]

wb = xl.load_workbook(new_bracket)
ws = wb['Games']

for i, row in enumerate(ROW_NUMS):
    ws[f"{TEAM_1_COL}{row}"] = team_1_col[i]
    ws[f"{TEAM_2_COL}{row}"] = team_2_col[i]
    ws[f"{SCORE_1_COL}{row}"] = region_scores_1[i]
    ws[f"{SCORE_2_COL}{row}"] = region_scores_2[i]
    ws[f"{EVAL_COL}{row}"] = region_analysis[i]

wb.save(new_bracket)